In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from scipy.fft import fft
from scipy.optimize import curve_fit
import os

data = pd.read_csv('Front_Back_E_C_Damp.csv')

In [ ]:
data['k'] = 1/(4*data['Height (m)'])

In [ ]:
samples_list = ['A', 'B', 'C']
no_of_samples = 4
which_bool = 'Front'

for i in range(len(samples_list)):
    start_index = 4 * (i)
    end_index = 4 * (i+1)
    
    x = data['k'].iloc[start_index:end_index]
    y = data['Natural Frequency '+which_bool+' (kHz)'].iloc[start_index:end_index]
    coefficients = np.polyfit(x, y, 1) 
    trendline = np.poly1d(coefficients)
    r_squared = np.corrcoef(x, y)[0, 1]**2
    plt.plot(np.linspace(0, 100), trendline(np.linspace(0, 100)), 'g--', label=f'Extended Line of Best Fit, $R^2={r_squared:.2f}$')

    plt.scatter(x, y, color='blue')

    
    plt.scatter(x, y)
    plt.title('Frequency against k for '+samples_list[i]+' ('+which_bool+')')
    plt.xlabel('k = 1/4H (m)')
    plt.ylabel('Frequency (kHz)')
    plt.ylim([0, 45])
    plt.xlim([0, 30])
    plt.legend()
    plt.savefig('Frequency against k for '+samples_list[i]+' ('+which_bool+')'+'.png')
    plt.show()

In [ ]:
data_averaged = data.copy(deep=True)
group_index = data_averaged.index // 4
data_averaged = data_averaged.drop(columns=data_averaged.columns[0])

# Calculate the average of each group for every column
data_averaged = data_averaged.groupby(group_index).mean()
lattice_types = data['Lattice'].iloc[::4].apply(lambda x: x[0])

# Assign these types to the averaged DataFrame
data_averaged['Lattice Type'] = lattice_types.values
cols = ['Lattice Type'] + [col for col in data_averaged if col != 'Lattice Type']
data_averaged = data_averaged[cols]
data_averaged

In [ ]:
for i, column in enumerate(data_averaged.columns):
    if column not in ['Unit Cell Size (mm)', 'Natural Frequency Front (kHz)', 'Height (m)', 'Relative Density', 'Natural Frequency Back (kHz)', 'Lattice Type', 'k']:
        
        x = data_averaged['Unit Cell Size (mm)']
        y = data_averaged[column]
        coefficients = np.polyfit(x, y, 1) 
        trendline = np.poly1d(coefficients)
        r_squared = np.corrcoef(x, y)[0, 1]**2
        plt.plot(np.linspace(0, 100), trendline(np.linspace(0, 100)), 'g--', label=f'Extended Line of Best Fit, $R^2={r_squared:.2f}$')
        
        plt.scatter(x, y)
        plt.title(f'{column} vs Unit Cell Size (mm)')
        plt.xlabel('Unit Cell Size (mm)')
        plt.ylabel(column)
        padding = 0.5
        padding_x = (x.max() - x.min()) * padding  # 5% padding
        padding_y = (y.max() - y.min()) * padding  # 5% padding
        plt.xlim([x.min() - padding_x, x.max() + padding_x])
        plt.ylim([y.min() - padding_y, y.max() + padding_y])
        plt.legend()
        safe_column_name = column.replace('/', ' per ').replace(' ', '_').replace('(', '').replace(')', '')
        plt.savefig(f'{safe_column_name} vs Unit Cell Size.png')
        plt.show()